## Clean IMDB : age data

**Source:** annotation_imdb_keras.py

In [ ]:
import importlib

In [ ]:
from scipy import io as spio
from datetime import datetime

import numpy as np
import os
import shutil

from utils import get_local_dataset_root_path, all_equal

In [ ]:
#-- added
import matplotlib.pyplot as plt
import PIL

In [ ]:
delimeter = ":\t"

# utils : stats
from scipy import stats

from utils_imdb import calc_age, is_valid, yob

In [ ]:
if(os.path.exists("./dataset/imdb_crop/")):
    DATASET_ROOT_PATH=""
else:
    DATASET_ROOT_PATH=get_local_dataset_root_path()

OUTPUT_LABEL="agegender_imdb"

IMDB_PATH=DATASET_ROOT_PATH+"dataset/imdb_crop/"

In [ ]:
meta = spio.loadmat(IMDB_PATH+"imdb.mat")

db = "imdb"

### Convert numpy ndarray to pandas DataFrame

Check
* All columns have same shape
* All rows of the column of type numpy.ndarray have same length (# elements)

In [ ]:
# column_names = ["full_path", "dob", "gender", "photo_taken", "face_score", "second_face_score"]
column_names = ["full_path", "dob", "gender", "photo_taken", "face_score", "second_face_score", "name"]

c_outer_type = np.array([isinstance(meta[db][0, 0][cn], np.ndarray) for cn in column_names])
c_inner_type = np.array([isinstance(meta[db][0, 0][cn][0], np.ndarray) for cn in column_names])
c_element_type = [type(meta[db][0, 0][cn][0][0]) for cn in column_names]

c_outer_shape = np.array([meta[db][0, 0][cn].shape for cn in column_names])
c_inner_shape = np.array([meta[db][0, 0][cn][0].shape for cn in column_names])

# print(c_outer_shape)
# print(c_inner_shape)
print("Consistent c_outer_shape ? \t", all_equal(c_outer_shape, c_outer_shape[0]))
print("Consistent c_inner_shape ? \t", all_equal(c_inner_shape, c_inner_shape[0]))

# print(c_outer_type)
# print(c_inner_type)
print("Consistent c_outer_type ? \t", all_equal(c_outer_type, True))
print("Consistent c_inner_type ? \t", all_equal(c_inner_type, True))

print(c_element_type)

for cn in column_names:
    if isinstance(meta[db][0, 0][cn][0][0], np.ndarray):
        col0 = meta[db][0, 0][cn][0]
        col0_len_element = np.array([len(element) for element in col0])
        print("Consistent element length of column: '%s' ? \t" % (cn), all_equal(col0_len_element, 1))

In [ ]:
import pandas as pd

df = pd.DataFrame()

for cn in column_names:
    if isinstance(meta[db][0, 0][cn][0][0], np.ndarray):
        df[cn] = [r[0] for r in meta[db][0, 0][cn][0] if len(r) == 1]
    else:
        df[cn] = meta[db][0, 0][cn][0]

In [ ]:
print("shape:\t", df.shape)
print(df)

In [ ]:
search_str = "nm0000001_rm980463616_1899-5-10_1968"

row = df.loc[df["full_path"].str.contains(search_str + ".")]
print(row)
print(row["face_score"].iloc[0])
print(type(row["face_score"]))

###  (Original code)

In [ ]:
full_path = meta[db][0, 0]["full_path"][0]
dob = meta[db][0, 0]["dob"][0]
gender = meta[db][0, 0]["gender"][0]
photo_taken = meta[db][0, 0]["photo_taken"][0]
face_score = meta[db][0, 0]["face_score"][0]
second_face_score = meta[db][0, 0]["second_face_score"][0]
age = [calc_age(photo_taken[i], dob[i]) for i in range(len(dob))]

In [ ]:
len(full_path)

In [ ]:
# get column names
print("column names:\n", meta[db][0, 0].dtype.names)

In [ ]:
name = meta[db][0, 0]["name"][0]
celeb_id = meta[db][0, 0]["celeb_id"][0]
celeb_names = meta[db][0, 0]["celeb_names"][0]

print(full_path.shape)
print(dob.shape)
print(name.shape)
print(celeb_id.shape)
print(celeb_names.shape)

# Note: celeb_names is from another table b/c # rows is less than that of full_path and dob
print([ii for ii, cid in enumerate(celeb_id) if cid >= celeb_names.shape[0]])

###  (New code)

In [ ]:
# find index of invalid images
invalid = []
valid = []
for i in range(len(full_path)):
    if(is_valid(face_score[i],second_face_score[i],age[i],gender[i])):
        valid.append(i)
    else:
        invalid.append(i)

In [ ]:
print("len(valid):", len(valid))
print("len(invalid):", len(invalid))

In [ ]:
# >> group valid index by folder
by_folder = {}

for folder in [d for d in os.listdir(IMDB_PATH) if not os.path.isfile(os.path.join(IMDB_PATH, d))]:
    by_folder[folder] = []
print(len(by_folder))

for vi in valid:
    parts = full_path[vi][0].split("/")
    if len(parts):
        by_folder[parts[0]].append(vi)

# print("key : # files")
# for key in by_folder.keys():
#     print(key, ":", len(by_folder[key]))

folder = 10

folder_str = "%02d" % (folder)
print("key : # files")
print(folder_str, ":", len(by_folder[folder_str]))

In [ ]:
# >> get all person id
len_nameid = 7
name_ids = []
# get nameid from all valid fullpath
for vi in valid:
    nameid = full_path[vi][0].split("/")[1][2:2+len_nameid]
    name_ids.append(int(nameid))
name_ids = np.array(name_ids)
unique_name_ids = np.unique(name_ids)

print(len(unique_name_ids))
print(unique_name_ids[:10])
print(unique_name_ids[-10:])

# np.unique(np.array([1,2,3,3,3,4,5,6,4]))

In [ ]:
#===========================================
## *** MORPH-II Cleaning process : age
# For each person whose birthdates differed <= 1 year (366 days to account for some leap years)
#     calculate mean birthdate
#     assign this date to all images
#===========================================

In [ ]:
# >> filter by person id
# pid = 93 # Brad Pitt
pid = 97 # Pamela Anderson
# pid = 98 # Jennifer Anniston
# pid = 6363555

if pid not in unique_name_ids:
    print("!! Warning !! : unique_name_ids does not contain pid %d" % pid)

filtered = [vi for vi in valid if "nm%07d_" % (pid) in full_path[vi][0]]
print("# images", delimeter, len(filtered))
# print([path[0] for path in full_path[filtered]])

f_name = np.array([name[fid][0] for fid in filtered])
f_yob = np.array([yob(dob[fid]) for fid in filtered])
if len(f_yob) < 20: print(f_yob)
# f_yob = np.array([dob[fid] for fid in filtered])

mode_result_name = stats.mode(f_name)
mode_name = mode_result_name.mode[0]
print("Name [most frequent]", delimeter, mode_name)

mode_result = stats.mode(f_yob)
mode = mode_result.mode[0]

# find majority vote
if all_equal(f_yob, mode):
    major_val = mode
    print("\nAll elements equal to Mode = %d" % (major_val))
else:
    major_val = f_yob.mean()
    print("\nMean = %.2f" % (major_val))

print("yob [majority vote]", delimeter, major_val)

In [ ]:
# # for each unique name id : check if all of yob are equal
# yob_major_values = []
# use_mode_yob = []

# ii = 0
# # for pid in unique_name_ids[:20]:
# for pid in unique_name_ids:
#     filtered = [vi for vi in valid if "nm%07d_" % (pid) in full_path[vi][0]]

#     f_name = np.array([name[fid][0] for fid in filtered])
#     f_yob = np.array([yob(dob[fid]) for fid in filtered])

#     mode_result_name = stats.mode(f_name)
#     mode_name = mode_result_name.mode[0]
# #     print("Name [most frequent]", delimeter, mode_name)

#     mode_result = stats.mode(f_yob)
#     mode = mode_result.mode[0]

#     # find majority vote
#     if all_equal(f_yob, mode):
#         major_val = mode
#         yob_all_equal = True
#     else:
#         major_val = f_yob.mean()
#         yob_all_equal = False
#     use_mode_yob.append(yob_all_equal)
#     yob_major_values.append(major_val)
#     if ii % 50 == 0:
#         print(ii)
#     ii += 1

In [ ]:
# print(all_equal(np.array(use_mode_yob), True))
# print([v for v in use_mode_yob if v])

### Show images (simple)

In [ ]:
# raw_index = 1751 #Brad Pitt
# raw_index = 3044 # Jennifer Aniston

# find raw_index in valid
v_ind = valid.index(raw_index)

print("v_ind:", v_ind)
print(valid[v_ind])

In [ ]:
%matplotlib inline

direction = +1 # forward
# direction = -1 # backward

step_size = 1
# step_size = 10

step = direction*step_size

ind = valid[v_ind]
print("\t\tindex" + delimeter, ind)

image = PIL.Image.open(os.path.join(IMDB_PATH, full_path[ind][0]))
plt.figure(figsize=(3.7,3.7))
plt.imshow(image)

if len(name[ind]) == 1:
    str_name = name[ind][0]
else:
    str_name = "More than 1 face ..."
print("name" + delimeter, str_name, '| ', celeb_names[celeb_id[ind]])

## print(celeb_id[ind])
## print('\t', celeb_names[celeb_id[ind]])
print("taken" + delimeter, photo_taken[ind])
print("yob" + delimeter, yob(dob[ind]))
print("age" + delimeter, calc_age(photo_taken[ind], dob[ind]))
v_ind = v_ind + step

In [ ]:
print("# images:\t", v_ind + 1)

### Show images

In [ ]:
#use the native Tk backend
%matplotlib tk

In [ ]:
import os
from utils import *
import utils_visualization as uvis
import DFInterface as dfi

In [ ]:
image_path = IMDB_PATH
print(image_path)

In [ ]:
# only valid images
show_filenames = [p[0] for p in full_path[valid]]

In [ ]:
importlib.reload(uvis)

In [ ]:
# start_vind = 3932
start_vind = 8191
start_index = valid.index(start_vind)
print(start_index)

(plot_batch_size, columns) = (8, 3)
fnit = uvis.FileNameIterator(image_path, show_filenames, plot_batch_size, columns, df, start_index)